#### barb_0MMPG_GF

In [ ]:
import time
from collections import namedtuple

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

import scipy.ndimage

try:
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.ticker as mticker

#from ipywidgets import *
try:
    import cv2
except:
    !pip install cv2
    import cv2
    
#import kernels
try:
    from skimage import io as skimage_io
except:
    !pip install scikit-image
    from skimage import io as skimage_io

try:
    import information_theory as IT
except:
    !pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"
    import information_theory as IT

from PIL import Image

import utils

In [ ]:
# apt install cm-super-minimal
# apt install dvipng
plt.rcParams.update({
    "text.usetex": True,
    #"font.family": "Helvetica",
    "font.family": "Serif",
    "text.latex.preamble": r"\usepackage{amsmath} \usepackage{amsfonts}"
})

In [ ]:
import logging
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
try:
    import denoising.image.gaussian as denoising
except:
    !pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
    import denoising.image.gaussian as denoising

In [ ]:
denoiser = denoising.Monochrome_Denoising(logger)

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

In [ ]:
sigma_kernel = 4.3
sigma = np.array([sigma_kernel, sigma_kernel])
kernel = [None]*2
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
print(np.sum(kernel[0]))
plt.plot(kernel[0])
plt.show()

In [ ]:
Args = namedtuple("args", ["input"])
args = Args("barb_0MMPG.png")

In [ ]:
try:
    Y = skimage_io.imread(args.input)
except FileNotFoundError:
    %run barb_0MMPG.ipynb
    Y = skimage_io.imread(args.input)

In [ ]:
logger.setLevel(logging.INFO)
denoised = denoiser.filter(Y, kernel)

In [ ]:
Y.shape

In [ ]:
Args = namedtuple("args", ["input"])
args = Args("barb.png")

In [ ]:
try:
    X = skimage_io.imread(args.input)
except FileNotFoundError:
    %run barb.ipynb
    X = skimage_io.imread(args.input)

In [ ]:
PSNR = IT.distortion.PSNR(denoised, X)

In [ ]:
STD_DEV = 40

In [ ]:
#plt.title(r"$\mathrm{GF}_{\text{" + str(sigma_kernel) + r"}}(\mathrm{Barb} + \mathbf{N}_{\mathcal{N}}), \mathbf{N}\sim \mathcal{N}_{}" + rf"(\mu=0, \sigma={std_dev})," + r"\mathrm{PSNR}=" + f"{PSNR:.2f}" + r"~\mathrm{dB}$")
string  = r"$"
string += r"\mathrm{GF}"
string += r"_\text{"
string += str(sigma_kernel)
string += r"}"
string += r"(\text{Barb} + \mathbf{N}_{\mathcal{N}(\sigma="
string += str(STD_DEV)
string += r")})"
string += r", \mathrm{PSNR}="
string += f"{PSNR:.2f}"
string += r"~\mathrm{dB}"
string += r"$"
plt.title(string)
plt.imshow(denoised, cmap="gray")
plt.savefig('barb_0MMPG_GF.pdf', bbox_inches='tight')

In [ ]:
pil_image = Image.fromarray(denoised.astype(np.uint8))
pil_image.save(f'barb_0MMPG_GF.png')